In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import copy
from tqdm import tqdm_notebook as tqdm

import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim


import argparse
from itertools import chain
import torch.nn.functional as F
from torch.autograd import Variable

import imageio
import scipy
from progressbar import ETA, Bar, Percentage, ProgressBar

import cv2
import numpy as np
import pandas as pd
# from scipy.misc import imresize
import scipy.io

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only. If you want to enable GPU, please to go Edit > Notebook Settings > Hardware Accelerator and select GPU.")

PyTorch Version:  1.4.0
Torchvision Version:  0.5.0
Using the GPU!


In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

# !ls /content/gdrive/Shared\ drives/CV\ Project\ Datasets/
!unzip -j /content/gdrive/Shared\ drives/CV\ Project\ Datasets/img_align_celeba.zip -d /content/DiscoGAN/datasets/celebA
!wget https://raw.githubusercontent.com/andrewliao11/CoGAN-tensorflow/master/list_attr_celeba.txt -P "/content/DiscoGAN/datasets/celebA"

%cd /content/DiscoGAN/

ModuleNotFoundError: No module named 'google'

In [17]:
#dataset_path = './datasets/'
dataset_path = 'datasets/'
celebA_path = os.path.join(dataset_path, 'celebA')
print(celebA_path)

def read_attr_file( attr_path, image_dir ):
    f = open( attr_path )
    lines = f.readlines()
    lines = list(map(lambda line: line.strip(), lines))
    columns = ['image_path'] + lines[1].split()
    lines = lines[2:]

    items = map(lambda line: line.split(), lines)
    df = pd.DataFrame( items, columns=columns )
    df['image_path'] = df['image_path'].map( lambda x: os.path.join( image_dir, x ) )

    return df

datasets/celebA


In [3]:
def read_images( filenames, domain=None, image_size=64):

    images = []
    for fn in filenames:
        image = cv2.imread(fn)
        if image is None:
            continue
        image = cv2.resize(image, (image_size,image_size))
        # image = image.astype(np.float16) / 255.
        image = image.astype(np.float32) / 255.
        image = image.transpose(2,0,1)
        images.append( image )

    images = np.stack( images )
    return images

In [4]:
def shuffle_data(da, db):
    a_idx = range(len(da))
    np.random.shuffle(list(a_idx))

    b_idx = range(len(db))
    np.random.shuffle(list(b_idx))

    shuffled_da = np.array(da)[ np.array(a_idx) ]
    shuffled_db = np.array(db)[ np.array(b_idx) ]

    return shuffled_da, shuffled_db


In [5]:
def as_np(data):
    return data.cpu().data.numpy()

In [6]:
def read_attr_file( attr_path, image_dir ):
    print(attr_path)
    print(image_dir)
    f = open( attr_path )
    lines = f.readlines()
    lines = list(map(lambda line: line.strip(), lines))
    columns = ['image_path'] + lines[1].split()
    lines = lines[2:]

    items = map(lambda line: line.split(), lines)
    df = pd.DataFrame( items, columns=columns )
    df['image_path'] = df['image_path'].map( lambda x: os.path.join( image_dir, x ) )

    return df

In [7]:
def get_celebA_files(style_A, style_B, constraint, constraint_type, test=False, n_test=200):
    attr_file = os.path.join( celebA_path, 'list_attr_celeba.txt' )
    image_dir = os.path.join( celebA_path, '' )
    image_data = read_attr_file( attr_file, image_dir )

    if constraint:
        image_data = image_data[ image_data[constraint] == constraint_type]

    style_A_data = image_data[ image_data[style_A] == '1']['image_path'].values
    if style_B:
        style_B_data = image_data[ image_data[style_B] == '1']['image_path'].values
    else:
        style_B_data = image_data[ image_data[style_A] == '-1']['image_path'].values

    if test == False:
        return style_A_data[:-n_test], style_B_data[:-n_test]
    if test == True:
        return style_A_data[-n_test:], style_B_data[-n_test:]

In [8]:
kernel_sizes = [4,3,3]
strides = [2,2,1]
paddings=[0,0,1]
latent_dim = 300

In [9]:
class Discriminator(nn.Module):
    def __init__(
            self,
            ):

        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 4, 2, 1, bias=False)
        self.relu1 = nn.LeakyReLU(0.2, inplace=True)

        self.conv2 = nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(64 * 2)
        self.relu2 = nn.LeakyReLU(0.2, inplace=True)

        self.conv3 = nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(64 * 4)
        self.relu3 = nn.LeakyReLU(0.2, inplace=True)

        self.conv4 = nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(64 * 8)
        self.relu4 = nn.LeakyReLU(0.2, inplace=True)

        self.conv5 = nn.Conv2d(64 * 8, 1, 4, 1, 0, bias=False)

    def forward(self, input):
        conv1 = self.conv1( input )
        relu1 = self.relu1( conv1 )

        conv2 = self.conv2( relu1 )
        bn2 = self.bn2( conv2 )
        relu2 = self.relu2( bn2 )

        conv3 = self.conv3( relu2 )
        bn3 = self.bn3( conv3 )
        relu3 = self.relu3( bn3 )

        conv4 = self.conv4( relu3 )
        bn4 = self.bn4( conv4 )
        relu4 = self.relu4( bn4 )

        conv5 = self.conv5( relu4 )

        return torch.sigmoid( conv5 ), [relu2, relu3, relu4]

In [10]:
class Generator(nn.Module):
    def __init__(
            self,
            extra_layers=False
            ):

        super(Generator, self).__init__()

        if extra_layers == True:
            self.main = nn.Sequential(
                nn.Conv2d(3, 64, 4, 2, 1, bias=False),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 2),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 4),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 8),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64 * 8, 100, 4, 1, 0, bias=False),
                nn.BatchNorm2d(100),
                nn.LeakyReLU(0.2, inplace=True),

                nn.ConvTranspose2d(100, 64 * 8, 4, 1, 0, bias=False),
                nn.BatchNorm2d(64 * 8),
                nn.ReLU(True),
                nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 4),
                nn.ReLU(True),
                nn.ConvTranspose2d(64 * 4, 64 * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 2),
                nn.ReLU(True),
                nn.ConvTranspose2d(64 * 2,     64, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(True),
                nn.ConvTranspose2d(    64,      3, 4, 2, 1, bias=False),
                nn.Sigmoid()
            )


        if extra_layers == False:
            self.main = nn.Sequential(
                nn.Conv2d(3, 64, 4, 2, 1, bias=False),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 2),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 4),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 8),
                nn.LeakyReLU(0.2, inplace=True),

                nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 4),
                nn.ReLU(True),
                nn.ConvTranspose2d(64 * 4, 64 * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 2),
                nn.ReLU(True),
                nn.ConvTranspose2d(64 * 2,     64, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(True),
                nn.ConvTranspose2d(    64,      3, 4, 2, 1, bias=False),
                nn.Sigmoid()
            )

    def forward(self, input):
        return self.main( input )

In [11]:
def get_fm_loss(real_feats, fake_feats, criterion):
    losses = 0
    for real_feat, fake_feat in zip(real_feats, fake_feats):
        l2 = (real_feat.mean(0) - fake_feat.mean(0)) * (real_feat.mean(0) - fake_feat.mean(0))
        loss = criterion( l2, Variable( torch.ones( l2.size() ) ).cuda() )
        losses += loss

    return losses

In [12]:
def get_gan_loss(dis_real, dis_fake, criterion, cuda):
    labels_dis_real = Variable(torch.ones( [dis_real.size()[0], 1] ))
    labels_dis_fake = Variable(torch.zeros([dis_fake.size()[0], 1] ))
    labels_gen = Variable(torch.ones([dis_fake.size()[0], 1]))
    
    #labels_dis_real = labels_dis_real.half()
    #labels_dis_fake = labels_dis_fake.half()
    #labels_gen = labels_gen.half()

    # if cuda:
    labels_dis_real = labels_dis_real.cuda()
    labels_dis_fake = labels_dis_fake.cuda()
    labels_gen = labels_gen.cuda()

    dis_loss = criterion( dis_real, labels_dis_real ) * 0.5 + criterion( dis_fake, labels_dis_fake ) * 0.5
    gen_loss = criterion( dis_fake, labels_gen )

    return dis_loss, gen_loss

In [13]:
def write_losses_to_file(loss_data):

  import csv
  import time

  timestr = time.strftime("%Y%m%d-%H%M%S")

  with open('loss_data_' + timestr + ".csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["epoch #", "GEN Loss A", "GEN Loss B", "FM Loss A", "FM Loss B", "RECON Loss A" , "RECON Loss B", "DIS Loss A", "DIS Loss B"])

    writer.writerows(loss_data)
    

In [26]:
# Default ARGS
imagesNum = 202599
args = type('', (), {})()

args.cuda = True
args.batch_size = 128
args.learning_rate = 0.0002
args.result_path = './results/'
args.model_path = './models/'
args.model_arch = 'discogan'
args.image_size = 64

args.gan_curriculum = 10000
args.starting_rate = 0.01
args.default_rate = 0.5

args.constraint = None
args.constraint_type = None
args.n_test = 200

args.update_interval = 3
args.log_interval = 234 / 2
args.image_save_interval = 10000
args.model_save_interval = 10000


# IMPORTANT ARGS
args.task_name = 'celebA'
args.style_A ='Male'
args.style_B =None
args.epoch_size = 1000   # originally 5000

args.write_losses_to_file = True



In [27]:
def image_translation(args):

  # cuda = args.cuda
  if args.cuda == True:
    cuda = True
  else:
    cuda = False
   
  task_name = args.task_name

  epoch_size = args.epoch_size    # 5000
  batch_size = args.batch_size    #  64

  result_path =  './results/celebA/Male'

  model_path = './models/celebA/Male'  

  data_style_A, data_style_B = get_celebA_files('Male', None, constraint=None, constraint_type=None, test=False, n_test=200)
  test_style_A, test_style_B = get_celebA_files('Male', None, constraint=None, constraint_type=None, test=True, n_test=200)

  ##_________________________Reduce training data_______________________
  data_style_A = data_style_A[:15000]
  data_style_B = data_style_B[:15000]

  # else:
  test_A = read_images( test_style_A, None, args.image_size )
  test_B = read_images( test_style_B, None, args.image_size )
  
  with torch.no_grad():
    #test_A = Variable( torch.HalfTensor( test_A ), volatile=True ).cuda()
    #test_B = Variable( torch.HalfTensor( test_B ), volatile=True ).cuda()
    test_A = Variable( torch.FloatTensor( test_A ), volatile=True ).cuda()
    test_B = Variable( torch.FloatTensor( test_B ), volatile=True ).cuda()
  # test_A = test_A.cuda()
  # test_B = test_B.cuda()


  if not os.path.exists(result_path):
    os.makedirs(result_path)
  if not os.path.exists(model_path):
    os.makedirs(model_path)

  generator_A = Generator()
  #generator_A = generator_A.half()
  generator_B = Generator()
  #generator_B = generator_B.half()
  generator_A = generator_A.cuda()
  generator_B = generator_B.cuda()

  discriminator_A = Discriminator()
  #discriminator_A = discriminator_A.half()
  discriminator_B = Discriminator()
  #discriminator_B = discriminator_B.half()
  discriminator_A = discriminator_A.cuda()
  discriminator_B = discriminator_B.cuda()
  '''
  for layer in generator_A.modules():
    if isinstance(layer, nn.BatchNorm2d):
        layer.float()
        
  for layer in generator_B.modules():
    if isinstance(layer, nn.BatchNorm2d):
        layer.float()
        
  for layer in discriminator_A.modules():
    if isinstance(layer, nn.BatchNorm2d):
        layer.float()
        
  for layer in discriminator_B.modules():
    if isinstance(layer, nn.BatchNorm2d):
        layer.float()
  '''
  data_size = min( len(data_style_A), len(data_style_B) )
  n_batches = ( data_size // batch_size )

  recon_criterion = nn.MSELoss()
  gan_criterion = nn.BCELoss()
  feat_criterion = nn.HingeEmbeddingLoss()

  gen_params = chain(generator_A.parameters(), generator_B.parameters())
  dis_params = chain(discriminator_A.parameters(), discriminator_B.parameters())

  optim_gen = optim.Adam( gen_params, lr=args.learning_rate, betas=(0.5,0.999), weight_decay=0.00001)
  optim_dis = optim.Adam( dis_params, lr=args.learning_rate, betas=(0.5,0.999), weight_decay=0.00001)

  iters = 0

  gen_loss_total = []
  dis_loss_total = []

  save_loss_data = []

  for epoch in range(epoch_size):
    data_style_A, data_style_B = shuffle_data( data_style_A, data_style_B)

    widgets = ['epoch #%d|' % epoch, Percentage(), Bar(), ETA()]
    pbar = ProgressBar(maxval=n_batches, widgets=widgets)
    pbar.start()

    for i in range(n_batches):

      pbar.update(i)

      generator_A.zero_grad()
      generator_B.zero_grad()
      discriminator_A.zero_grad()
      discriminator_B.zero_grad()

      A_path = data_style_A[ i * batch_size: (i+1) * batch_size ]
      B_path = data_style_B[ i * batch_size: (i+1) * batch_size ]

      A = read_images( A_path, None, args.image_size )
      B = read_images( B_path, None, args.image_size )

      #A = Variable( torch.HalfTensor( A ) )
      #B = Variable( torch.HalfTensor( B ) )
      A = Variable( torch.FloatTensor( A ) )
      B = Variable( torch.FloatTensor( B ) )

      A = A.cuda()
      B = B.cuda()

      AB = generator_B(A)
      BA = generator_A(B)

      ABA = generator_A(AB)
      BAB = generator_B(BA)

      # Reconstruction Loss
      recon_loss_A = recon_criterion( ABA, A )
      recon_loss_B = recon_criterion( BAB, B )

      # Real/Fake GAN Loss (A)
      A_dis_real, A_feats_real = discriminator_A( A )
      A_dis_fake, A_feats_fake = discriminator_A( BA )

      dis_loss_A, gen_loss_A = get_gan_loss( A_dis_real, A_dis_fake, gan_criterion, cuda )
      fm_loss_A = get_fm_loss(A_feats_real, A_feats_fake, feat_criterion)

      # Real/Fake GAN Loss (B)
      B_dis_real, B_feats_real = discriminator_B( B )
      B_dis_fake, B_feats_fake = discriminator_B( AB )

      dis_loss_B, gen_loss_B = get_gan_loss( B_dis_real, B_dis_fake, gan_criterion, cuda )
      fm_loss_B = get_fm_loss( B_feats_real, B_feats_fake, feat_criterion )

      # Total Loss

      if iters < args.gan_curriculum:
        rate = args.starting_rate
      else:
        rate = args.default_rate

      gen_loss_A_total = (gen_loss_B*0.1 + fm_loss_B*0.9) * (1.-rate) + recon_loss_A * rate
      gen_loss_B_total = (gen_loss_A*0.1 + fm_loss_A*0.9) * (1.-rate) + recon_loss_B * rate
      
      #generative_loss= []
      #discriminative_loss=[]

      gen_loss = gen_loss_A_total + gen_loss_B_total
      dis_loss = dis_loss_A + dis_loss_B

      #generative_loss.append(gen_loss)
      #discriminative_loss.append(dis_loss)

      if iters % args.update_interval == 0:    
        dis_loss.backward()
        optim_dis.step()
      else:
        gen_loss.backward()
        optim_gen.step()

      if iters % args.log_interval == 0:
        print("---------------------")
        print("GEN Loss:", as_np(gen_loss_A.mean()), as_np(gen_loss_B.mean()))
        print("Feature Matching Loss:", as_np(fm_loss_A.mean()), as_np(fm_loss_B.mean()))
        print("RECON Loss:", as_np(recon_loss_A.mean()), as_np(recon_loss_B.mean()))
        print("DIS Loss:", as_np(dis_loss_A.mean()), as_np(dis_loss_B.mean()))

        if args.write_losses_to_file == True:
          loss_list = [str(epoch) , as_np(gen_loss_A.mean()) , as_np(gen_loss_B.mean()) \
                        , as_np(fm_loss_A.mean()) , as_np(fm_loss_B.mean()) \
                        , as_np(recon_loss_A.mean()) , as_np(recon_loss_B.mean()) \
                        , as_np(dis_loss_A.mean()) , as_np(dis_loss_B.mean())]

          save_loss_data.append(loss_list)

      if iters % args.image_save_interval == 0:
        AB = generator_B( test_A )
        BA = generator_A( test_B )
        ABA = generator_A( AB )
        BAB = generator_B( BA )

        n_testset = 5                                                             #min( test_A.size()[0], test_B.size()[0] )

        subdir_path = os.path.join( result_path, str(iters / args.image_save_interval) )

        if os.path.exists( subdir_path ):
          pass
        else:
          os.makedirs( subdir_path )

        for im_idx in range( n_testset ):
          A_val = test_A[im_idx].cpu().data.numpy().transpose(1,2,0) * 255.
          B_val = test_B[im_idx].cpu().data.numpy().transpose(1,2,0) * 255.
          BA_val = BA[im_idx].cpu().data.numpy().transpose(1,2,0)* 255.
          ABA_val = ABA[im_idx].cpu().data.numpy().transpose(1,2,0)* 255.
          AB_val = AB[im_idx].cpu().data.numpy().transpose(1,2,0)* 255.
          BAB_val = BAB[im_idx].cpu().data.numpy().transpose(1,2,0)* 255.

          filename_prefix = os.path.join (subdir_path, str(im_idx))
          imageio.imwrite( filename_prefix + '.A.jpg', A_val.astype(np.uint8)[:,:,::-1])
          imageio.imwrite( filename_prefix + '.B.jpg', B_val.astype(np.uint8)[:,:,::-1])
          imageio.imwrite( filename_prefix + '.BA.jpg', BA_val.astype(np.uint8)[:,:,::-1])
          imageio.imwrite( filename_prefix + '.AB.jpg', AB_val.astype(np.uint8)[:,:,::-1])
          imageio.imwrite( filename_prefix + '.ABA.jpg', ABA_val.astype(np.uint8)[:,:,::-1])
          imageio.imwrite( filename_prefix + '.BAB.jpg', BAB_val.astype(np.uint8)[:,:,::-1])

      if iters % args.model_save_interval == 0:
        torch.save( generator_A, os.path.join(model_path, 'model_gen_A-' + str( iters / args.model_save_interval )))
        torch.save( generator_B, os.path.join(model_path, 'model_gen_B-' + str( iters / args.model_save_interval )))
        torch.save( discriminator_A, os.path.join(model_path, 'model_dis_A-' + str( iters / args.model_save_interval )))
        torch.save( discriminator_B, os.path.join(model_path, 'model_dis_B-' + str( iters / args.model_save_interval )))

      iters += 1

  if args.write_losses_to_file == True:
    write_losses_to_file(save_loss_data)

In [28]:
image_translation(args)

C:\Users\tamim\AppData\Local\Continuum\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\tamim\AppData\Local\Continuum\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
epoch #0|N/A%|                                                  |ETA:  --:--:--

---------------------
GEN Loss: 0.6317123 0.8984958
Feature Matching Loss: 0.34615022 0.375065
RECON Loss: 0.14104351 0.11759075
DIS Loss: 0.6810744 0.7383685


epoch #1|  0%|                                                  |ETA:   0:00:54

---------------------
GEN Loss: 0.56296057 0.81385565
Feature Matching Loss: 0.044256095 0.03207411
RECON Loss: 0.04635529 0.04452808
DIS Loss: 0.69313604 0.70670927


epoch #2|N/A%|                                                  |ETA:  --:--:--

...

epoch #998|  0%|                                                |ETA:   0:00:52

---------------------
GEN Loss: 6.573288 5.8552175
Feature Matching Loss: 0.017353646 0.015097278
RECON Loss: 0.01618284 0.014064379
DIS Loss: 0.004474528 0.013345217


epoch #999|  0%|                                                |ETA:   0:00:54

---------------------
GEN Loss: 6.866289 5.6940103
Feature Matching Loss: 0.018245157 0.014616625
RECON Loss: 0.015689246 0.013574613
DIS Loss: 0.0032857093 0.011683319


epoch #999| 99%|############################################### |ETA:   0:00:00